On the sample data, build up a Jupyter file to transform the data into the desired format of modeling.

In [1]:
# Import modules
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors 

In [2]:
%run ./Repos-exe-sample.ipynb

./Repos-exe-sample.ipynb:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "source": [
./Repos-exe-sample.ipynb:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "!pip install seaborn\n",
./Repos-exe-sample.ipynb:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "import pandas as pd\n",
./Repos-exe-sample.ipynb:11: Sett

In [3]:
df = pd.concat([df_mix,df_rh])
for col in ['author_name','author_email','author_date','author_affiliation','auth_comm','diff_date']:
    del df[col]

In [4]:
df.columns

Index(['repos_id0', 'committer_name', 'committer_email', 'committer_date',
       'added', 'removed', 'whitespace', 'filename', 'date_attempted', 'git',
       'repos_name', 'committer_affiliation', 'committer_year',
       'committer_month', 'committer_day'],
      dtype='object')

In [58]:
#Extract the email name
df["email_name"]=df['committer_email'].str.split('@', 1, expand=True)[0]

## Filter out observations used as test set:
- People using two email address
- People in rh employee list

#### People using two email address

In [59]:
committer_two = []
for ele in df['committer_name'].unique():
    if len(df[df['committer_name']==ele]['email_name'].unique())>1:
        committer_two.append(ele)

In [60]:
df_test = df[df['committer_name'].isin(committer_two)]

#### People in rh employee list

In [61]:
rh_txt=list(open('rh-employee-list.txt','r').read().split('\n'))

In [62]:
!pip install unidecode

In [63]:
# Transfer names in github data from Unicode to ACSII
# https://en.wikipedia.org/wiki/List_of_Unicode_characters#Basic_Latin
from unidecode import unidecode
name_df={'name_p': df['committer_name'][df['committer_affiliation']=='RH/V'].unique()}
name_df=pd.DataFrame(name_df)

In [64]:
res=[]
for ele in df['committer_name'][df['committer_affiliation']=='RH/V'].unique():
    res.append(unidecode(ele))
name_df['name_a']=res

In [65]:
print(sum(name_df['name_a'].isin(rh_txt)),sum(name_df['name_p'].isin(rh_txt)))

183 172


In [66]:
df_test2 = df[df['committer_name'].isin(name_df['name_a'][name_df['name_a'].isin(rh_txt)])]

#### Combine these two sets into a 'test set' and delete these observations from df

In [67]:
df_testset = pd.concat([df_test,df_test2]).drop_duplicates()

In [68]:
df_trainset = df.drop(df_testset.index)

## 1. Percentage of commit made on each day / each part of day

In [69]:
# Feature of weekday
day=[]
for ele in df['committer_date']: #weekday=0, weekend=1
    day.append(ele.weekday())
df['weekday']=day

In [70]:
# Feature of morning or evening
def get_part_of_day(hour):
    return (
        "morning" if 5 <= hour <= 11
        else
        "afternoon" if 12 <= hour <= 17
        else
        "evening" if 18 <= hour <= 22
        else
        "night"
    )

res=[]
for ele in df['committer_date']:
    res.append(get_part_of_day(ele.hour))
df['day_part']=res

In [71]:
#A df counting how many commits from weekday/weekend for each committer
weekday_df=df.groupby(['committer_name','weekday'])['committer_affiliation'].count().reset_index(name='count')
#A df counting how many commits from each part of day for each committer
daypart_df=df.groupby(['committer_name','day_part'])['committer_affiliation'].count().reset_index(name='count')
#Calculate what percentage a committer commit in weekday/weekend
committer_weekday=weekday_df.groupby(['committer_name','weekday']).agg({'count':'sum'})
committer_weekday=committer_weekday.groupby(level=0).apply(lambda x: x/float(x.sum()))
#Calculate what percentage a committer commit in each part of day
committer_daypart=daypart_df.groupby(['committer_name','day_part']).agg({'count':'sum'})
committer_daypart=committer_daypart.groupby(level=0).apply(lambda x: x/float(x.sum()))

In [72]:
#Reframing the weekday dataframe 
committer_weekday=committer_weekday.pivot_table(columns='weekday', index='committer_name', values='count', fill_value=0)
#Reframing the daypart dataframe 
committer_daypart=committer_daypart.pivot_table(columns='day_part', index='committer_name', values='count', fill_value=0)

In [73]:
new_df = pd.concat([committer_weekday,committer_daypart],axis=1)

## 2. Committer's contribution features

The creation of features are on project level. If people have participated in multiple projects, they will be added up, for % of changes.

#### 2.1 Yearly Rank and project count

In [74]:
def project_year(ele): 
    subdf=df[df['repos_name']==ele]
    year_df=subdf.groupby(['committer_year','committer_name'])['committer_affiliation'].count().reset_index(name='count')
    #Rank column
    year_df['rank_yr']=year_df.groupby(['committer_year'])['count'].rank(ascending=False).astype(int)
    year_df = year_df.groupby('committer_name')['rank_yr'].sum()
    return(year_df)

In [75]:
res = project_year('camel') #use the first one to set storage format
project_name = list(df['repos_name'].unique())

In [76]:
for ele in project_name[1:]: #delete camel because already used as storage format
    res=res.append(project_year(ele))    

In [77]:
# Edit format 
res['name']=res.index
res = pd.DataFrame(res)

In [78]:
# Summarize the rank and append to new_df
count = res.groupby('committer_name').size()
res = res.groupby('committer_name')['rank_yr'].sum()
res = pd.DataFrame(res)
res['count'] = count

In [79]:
new_df = pd.concat([new_df,res],axis=1)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


#### 2.2 Sum of percentage of changes in each project 

In [80]:
#for ele in df['repos_name'].unique():
def project_df(ele): #ele is the project name
    #Data frame for this project
    df_proj=df[df['repos_name']==ele]
    df_proj2=df_proj.groupby('committer_name')['added','removed','whitespace'].sum()
    df_proj2=pd.merge(df_proj2,df_proj.groupby('committer_name').size().reset_index(name='touched'),on='committer_name')
    
    #Transfer number of changes to percentage / total number in this project
    df_proj2['added']=df_proj2['added']/sum(df_proj2['added'])
    df_proj2['removed']=df_proj2['removed']/sum(df_proj2['removed'])
    df_proj2['whitespace']=df_proj2['whitespace']/sum(df_proj2['whitespace'])
    df_proj2['touched']=df_proj2['touched']/sum(df_proj2['touched'])
    return(df_proj2)

In [81]:
res = project_df('camel') #use the first one to set storage format
for ele in project_name[1:]: #delete camel because already used as storage format
    res=res.append(project_df(ele))  

In [82]:
res = res.groupby('committer_name')['added','removed','whitespace','touched'].sum()

In [83]:
new_df = pd.concat([new_df, res], axis=1)
# NaN means 0 of changes so no percentage

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


## 3. Nearest neighbor

In [84]:
def knn_committer(df,ele): #df: original; ele: project name
    
    
    
    #filter out repos with count>2
    if len(df[df['repos_name']==ele].groupby('committer_name'))>2: 
        
        knn_df=df[df['repos_name']==ele]
        knn_df=knn_df.groupby('committer_name')['added','removed','whitespace'].sum()
        
        #normalized the dataframe
        for i in ['added','removed','whitespace']:
            knn_df[i]=(knn_df[i]-min(knn_df[i]))
            
        knn_df.fillna(0)
        
        #find the nearest neighbor
        nbrs=NearestNeighbors(n_neighbors=2,algorithm='ball_tree').fit(knn_df)
        distances, indices = nbrs.kneighbors(knn_df)
        res=[]
        for i in range(len(knn_df['added'])):
            res.append(indices[i][1])
        knn_df['neighbor']=res
        
        knn_df['committer_name']=knn_df.index
        
        #Append neighbors' names as a new column
        res=[]
        names=[]
        #some of the first items in indices are the committer themselves, some are not. Need a if-else to filter out
        for i in range(len(indices)):
            if indices[i][0]==i:
                res.append(indices[i][1])
            else:
                res.append(indices[i][0]) #save the neighbor other than itself
            names.append(knn_df['committer_name'][res[i]])
        knn_df['neighbor_name']=names
        
    else: 
        
        knn_df=df[df['repos_name']==ele]
        knn_df=knn_df.groupby('committer_name')['added','removed','whitespace'].sum()
        
        #standardize the dataframe
        for i in ['added','removed','whitespace']:
            knn_df[i]=(knn_df[i]-np.mean(knn_df[i]))
            
        knn_df.fillna(0)
        
        knn_df['neighbor']=['Na']*len(knn_df)
        knn_df['committer_name']=knn_df.index
        knn_df['neighbor_name']=['Na']*len(knn_df)
        
    return(knn_df)
        

In [85]:
# Iterate for each project
ans = knn_committer(df, 'camel')

for ele in project_name[1:]:
    tmp = knn_committer(df.fillna(0),ele)
    ans = ans.append(tmp)

In [86]:
# Add a column to indicate the neighbor's affiliation
res = []

for ele in ans['neighbor_name']:
    if ele != 'Na':
        compare_tmp=df['committer_affiliation'][df['committer_name']==ele]
        res.append(compare_tmp.iloc[0])
    else:
        res.append('Na')
ans['affiliation']=res

In [87]:
# Numerize the affiliation column for group by 
ans['affiliation']=np.where(ans['affiliation']=='RH/V', 0, 1)

In [88]:
new_df['affiliation']=ans.groupby('committer_name')['affiliation'].sum()
# The affiliation column indicates how many neighbor of this committer is from Red Hat.

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'committer_name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


## 4. Encoding for repos name

In [89]:
encoding_tmp = pd.DataFrame(df.groupby(['committer_name','repos_name']).count()['repos_id0'])

In [90]:
encoding_tmp

,,repos_id0
committer_name,repos_name,
"""Caius """"kaio"""" Chance""",zanata-platform,1
--add,keycloak-documentation,1
14rcole,ostree-go,1
Aaron Patterson,rails,8
Aaron Prindle,minikube,2
Aaron Tan,docutils,7
Abhijith Das,resource-agents,2
Abhishek Koneru,pki,1
Abhishek Shah,kubernetes,1


In [91]:
encoding_tmp = encoding_tmp.pivot_table(columns = 'repos_name', index = 'committer_name', values = 'repos_id0', fill_value = 0)

In [92]:
new_df = pd.concat([new_df,encoding_tmp],axis=1)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [93]:
new_df.to_csv('transformed_sample.csv')